# Modelling a centrifugal compressor


In [1]:
from OCC.Core.BRepTools import breptools_Read
from OCC.Core.TopoDS import *
from OCC.Core.BRep import BRep_Builder
from OCC.Core.gp import *
from OCC.Core.BRepBuilderAPI import *
from math import pi,degrees,radians
from OCC.Core.BRepTools import breptools_Write
import OCCUtils
import OCCUtils.edge
#import numpy as np
import OCCUtils.face
from OCC.Core.BRepAlgoAPI import *
from OCC.Core.BRepMesh import BRepMesh_IncrementalMesh
from OCC.Core.BRepAdaptor import *
#import HackerCAD
from OCC.Core.GCE2d import *
from OCC.Core.Geom import *
from OCC.Core.BRepPrimAPI import *

from OCC.Display.WebGl.jupyter_renderer import JupyterRenderer

In [2]:
# Here we load the root_face. The root face is the face where the root
# each blade is attached. This face was created in FreeCAD and is loaded
# here. It is a revolved b-spline.
file_shape = TopoDS_Shape()
builder = BRep_Builder()
assert breptools_Read(file_shape,"./root_surface.brep",builder)
root_face = OCCUtils.Topo(file_shape).faces().__next__()

In [3]:
renderer = JupyterRenderer()

In [4]:
renderer.DisplayShape(root_face)

In [9]:
renderer

Here, the root surface is inspected.

In [10]:
bas = BRepAdaptor_Surface(root_face)

In [11]:
bas.FirstUParameter(), bas.LastUParameter()

(0.0, 6.28318530717959)

In [12]:
bas.FirstVParameter(), bas.LastVParameter()

(0.0, 1.0)

In [13]:
bas.IsUPeriodic()

True

In [14]:
bas.IsVPeriodic()

False

### Drawing blade root line on the blade root surface

Next I am trying to create an edge on root_face that will be the edge where the compressor blade meets the blade root surface.  In FreeCAD I drew a planar face, and then drew a b-spline on the face.  I will compute the p-curve of the b-spline on the planar face, and then use that p-curve on the the blade root_face to make the blade root edge. As you will see below, I am able to compute the p-curve, but have not been able to make the edge on the blade root face.

In [15]:
# Planar face
ctl_domain_brep_file_contents = TopoDS_Shape()
builder = BRep_Builder()
assert breptools_Read(ctl_domain_brep_file_contents,"./ctl_domain.brep",builder)
face_ctl_domain = OCCUtils.Topo(ctl_domain_brep_file_contents).faces().__next__()
del ctl_domain_brep_file_contents

In [16]:
renderer.DisplayShape(face_ctl_domain)

In [17]:
# b-spline on planar face
file_contents = TopoDS_Shape()
builder = BRep_Builder()
assert breptools_Read(file_contents,"./root_curve_on_ctl_domain.brep",builder)
edge_root_curve_3d_on_ctl_domain = OCCUtils.Topo(file_contents).edges().__next__()
del file_contents

In [18]:
bas_ctl_domain = BRepAdaptor_Surface(face_ctl_domain)

In [19]:
bas_ctl_domain.FirstUParameter(),bas_ctl_domain.LastUParameter()

(-200.0, 200.0)

In [20]:
bas_ctl_domain.FirstVParameter(),bas_ctl_domain.LastVParameter()

(-50.0, 50.0)

In [21]:
bas_ctl_domain.Value(0,0)

< gp_Pnt: 200.0, 50.0, 0.0 >

In [22]:
bac = BRepAdaptor_Curve(edge_root_curve_3d_on_ctl_domain,face_ctl_domain)

In [23]:
cos = bac.CurveOnSurface()

In [24]:
hcurve = cos.GetCurve()

In [25]:
hcurve.__class__

OCC.Core.Adaptor2d.Adaptor2d_HCurve2d

In [26]:
gas = bas.Surface()
gas.__class__

OCC.Core.GeomAdaptor.GeomAdaptor_Surface

In [27]:
surf = bas.Surface().Surface()

In [28]:
surf

<class 'Geom_Surface'>

In [29]:
mf = BRepBuilderAPI_MakeFace(hcurve,bas)

NotImplementedError: Wrong number or type of arguments for overloaded function 'new_BRepBuilderAPI_MakeFace'.
  Possible C/C++ prototypes are:
    BRepBuilderAPI_MakeFace::BRepBuilderAPI_MakeFace()
    BRepBuilderAPI_MakeFace::BRepBuilderAPI_MakeFace(TopoDS_Face const &)
    BRepBuilderAPI_MakeFace::BRepBuilderAPI_MakeFace(gp_Pln const &)
    BRepBuilderAPI_MakeFace::BRepBuilderAPI_MakeFace(gp_Cylinder const &)
    BRepBuilderAPI_MakeFace::BRepBuilderAPI_MakeFace(gp_Cone const &)
    BRepBuilderAPI_MakeFace::BRepBuilderAPI_MakeFace(gp_Sphere const &)
    BRepBuilderAPI_MakeFace::BRepBuilderAPI_MakeFace(gp_Torus const &)
    BRepBuilderAPI_MakeFace::BRepBuilderAPI_MakeFace(opencascade::handle< Geom_Surface > const &,Standard_Real const)
    BRepBuilderAPI_MakeFace::BRepBuilderAPI_MakeFace(gp_Pln const &,Standard_Real const,Standard_Real const,Standard_Real const,Standard_Real const)
    BRepBuilderAPI_MakeFace::BRepBuilderAPI_MakeFace(gp_Cylinder const &,Standard_Real const,Standard_Rea